# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json

# Import API key
from api_keys import g_key
from pprint import pprint


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Set up lists to store retrieved data from Open Weather
City_ID=[]
City=[]
Cloudiness=[]
Country=[]
Date=[]
Humidity=[]
Lat=[]
Lng=[]
MaxTemp=[]
WindSpeed=[]

In [3]:
#  ["City_ID","City","Cloudiness","Country","Date","Humidity","Lat","Lng","Max Temp","Wind Speed"]
# Open the CSV
# Set path for file
csvpath = os.path.join("..","Output_data","cities.csv")
#print(csvpath)
with open(csvpath,"r",encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=",")
    csv_header = next(csvreader)
    print(f"CSV Header: {csv_header}")
    csvreader = csv.reader(csvfile, delimiter=",")
    cnt=0
    for row in csvreader:
        City_ID.append(row[0])
        City.append(row[1])
        Cloudiness.append(row[2])
        Country.append(row[3])
        Date.append(row[4])
        Humidity.append(row[5])
        Lat.append(row[6])
        Lng.append(row[7])
        MaxTemp.append(row[8])
        WindSpeed.append(row[9])
cities_data =zip(City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,MaxTemp,WindSpeed)
cities_data_df = pd.DataFrame(cities_data, columns=("City_ID","City","Cloudiness","Country","Date","Humidity","Lat","Lng","Max Temp","Wind Speed"))     
cities_data_df
        

CSV Header: ['City_ID', 'City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat', 'Lng', 'Max Temp', 'Wind Speed']


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,6541260,Valenzano,0,IT,1631426312,83,41.0453,16.8851,69.31,1.99
1,3654055,Montecristi,90,EC,1631426314,83,-1.05,-80.6667,71.42,10.36
2,461699,Sasovo,98,RU,1631426178,80,54.3537,41.9199,62.76,4.85
3,1938756,Pante Macassar,81,ID,1631426316,55,-9.2028,124.3684,88.02,4.52
4,2948598,Birkenfeld,35,DE,1631426316,89,48.8667,8.6333,56.91,2.13
...,...,...,...,...,...,...,...,...,...,...
558,733638,Ardino,27,BG,1631426929,37,41.5833,25.1333,52.23,3.8
559,618067,Chiţcani,100,MD,1631426930,42,46.783,29.6168,68.52,6.38
560,2975408,Scionzier,5,FR,1631426931,88,46.0545,6.5576,62.22,2.66
561,2773447,Krieglach,20,AT,1631426933,79,47.5333,15.55,57.56,1.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [75]:
# Configure gmaps
gmaps.configure(api_key=g_key)
all_locations = cities_data_df[["Lat", "Lng"]].astype(float)
all_humidity = cities_data_df["Humidity"].astype(float)
#humidity.value_counts()


In [76]:
# Plot Heatmap
#fig = gmaps.figure()
#markers = gmaps.marker_layer(locations)
#fig.add_layer(markers)
#fig

In [77]:
# Plot Heatmap
# Create heat layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(all_locations, weights=all_humidity,
                                 dissipating=False, max_intensity=95,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:

# ['City_ID', 'City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat', 'Lng', 'Max Temp', 'Wind Speed']
ideal_weather_cities_df = cities_data_df.loc[(cities_data_df['Max Temp'].astype(float) > 70) 
                                         & (cities_data_df['Max Temp'].astype(float) < 80)
                                         & (cities_data_df['Wind Speed'].astype(float) < 10)
                                         & (cities_data_df['Cloudiness'].astype(float) == 0)]
# remove any rows with null values
ideal_weather_cities_df = ideal_weather_cities_df.dropna(how="any")
ideal_weather_cities_df



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
44,139889,Būkān,0,IR,1631426356,17,36.521,46.2089,79.83,3.67
64,261507,Kalymnos,0,GR,1631426380,64,36.95,26.9833,72.93,9.22
68,5116049,East Rockaway,0,US,1631426385,78,40.6421,-73.6696,71.15,4
86,5222680,Hopkinton,0,US,1631426407,85,41.4612,-71.7776,70.52,8.75
91,3995050,Nava,0,MX,1631426221,30,28.4167,-100.7667,75.99,6.78
144,4699626,Hutto,0,US,1631426472,66,30.5427,-97.5467,76.57,3
152,3174658,Provincia di Livorno,0,IT,1631426480,61,43.2333,10.5833,72.68,3.89
207,116402,Darjazin,0,IR,1631426548,27,35.6539,53.338,79.97,6.49
231,4115412,Hot Springs,0,US,1631426317,72,34.5037,-93.0552,72.68,2.71
337,6537550,Giovinazzo,0,IT,1631426680,82,41.1864,16.6674,70.09,4


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [67]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFram

type_of_place="hotel"
place_name="Hotel"
hotel_df = ideal_weather_cities_df
hotel_df["Hotel Name"]=""
hotel_df.iloc[0,7]

for i in range(len(hotel_df)):
    Lat=hotel_df.iloc[i,6]
    Lon=hotel_df.iloc[i,7]
    query_url = (f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={Lat},{Lon}&radius=5000&types={type_of_place}&name={place_name}&key={g_key}")
    hotel_response = requests.get(query_url).json()
    try:
        #hotel_response['results']
        #pprint(hotel_response['results'])
        data=hotel_response['results']
        print(f"City is: {hotel_df.iloc[i,1]}, Hotel: {data[0]['name']}")
        hotel_df.iloc[i,10]=data[0]["name"]
    except IndexError:
        print(f"*No Hotel found for: {hotel_df.iloc[i,1]}")
#pprint(data['name'])

City is: Būkān, Hotel: The Izadi Hotel
City is: Kalymnos, Hotel: Kalydna Island Hotel
City is: East Rockaway, Hotel: Rockville Centre Hotel
*No Hotel found for: Hopkinton
City is: Nava, Hotel: Hotel Villaloreto
City is: Hutto, Hotel: Hampton Inn & Suites Hutto Austin
City is: Provincia di Livorno, Hotel: Park Hotel Marinetta
City is: Darjazin, Hotel: Pasargad Hotel Darjazin Semnan
City is: Hot Springs, Hotel: Arlington Resort Hotel & Spa
City is: Giovinazzo, Hotel: Riva Del Sole Hotel
City is: Yosemite Lakes, Hotel: Black Hawk Lodge
*No Hotel found for: Răcari
City is: Cercola, Hotel: Hotel San Mauro
City is: Damanhur, Hotel: AL Tayser Hotel فندق التيسير
City is: Adra, Hotel: Hostal Zapata
City is: Sagunto, Hotel: Hotel AGH Canet


In [68]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
44,139889,Būkān,0,IR,1631426356,17,36.521,46.2089,79.83,3.67,The Izadi Hotel
64,261507,Kalymnos,0,GR,1631426380,64,36.95,26.9833,72.93,9.22,Kalydna Island Hotel
68,5116049,East Rockaway,0,US,1631426385,78,40.6421,-73.6696,71.15,4,Rockville Centre Hotel
86,5222680,Hopkinton,0,US,1631426407,85,41.4612,-71.7776,70.52,8.75,
91,3995050,Nava,0,MX,1631426221,30,28.4167,-100.7667,75.99,6.78,Hotel Villaloreto
144,4699626,Hutto,0,US,1631426472,66,30.5427,-97.5467,76.57,3,Hampton Inn & Suites Hutto Austin
152,3174658,Provincia di Livorno,0,IT,1631426480,61,43.2333,10.5833,72.68,3.89,Park Hotel Marinetta
207,116402,Darjazin,0,IR,1631426548,27,35.6539,53.338,79.97,6.49,Pasargad Hotel Darjazin Semnan
231,4115412,Hot Springs,0,US,1631426317,72,34.5037,-93.0552,72.68,2.71,Arlington Resort Hotel & Spa
337,6537550,Giovinazzo,0,IT,1631426680,82,41.1864,16.6674,70.09,4,Riva Del Sole Hotel


In [73]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [98]:
#hover_text=hotel_df[["Hotel Name"]].values.tolist()
#print(hover_text)

In [101]:
# Add marker layer ontop of heat map
# Display figure

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(all_locations, weights=all_humidity,
                                 dissipating=False, max_intensity=95,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig
# Add layer for extra labels for Hotel Name, City and Country

markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [105]:
from ipywidgets.embed import embed_minimal_html
embed_minimal_html("../Output_data/City_Humidity_Hotel_Heatmap.html", views=[fig]);